Building a simple cnn Arcchitecture

In [1]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# --- Define Model Parameters ---
# Let's assume our images will be resized to 64x64 pixels with 3 color channels (RGB)
IMG_HEIGHT = 64
IMG_WIDTH = 64
# Let's assume we have 10 fashion categories to predict
NUM_CLASSES = 10

# --- Build the CNN Model ---
model = keras.Sequential([
    # Input Layer: Specify the shape of our images
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    
    # First Convolutional Block 
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Second Convolutional Block
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Flatten the 2D feature maps into a 1D vector
    layers.Flatten(),
    
    # A standard Dense layer for classification
    layers.Dense(128, activation='relu'),
    
    # Output Layer: NUM_CLASSES neurons with softmax for multi-class probability
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# Print the model's summary to see how the data shape changes
model.summary()




Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,626,442 (6.20 MB)

 Trainable params: 1,626,442 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

Formular for Convolutional layer = (input - kernel size + 1)
Formular for Max pooling layer = floor(input/pool size)

#### CNN Image classification

In [2]:
import os
import zipfile
import shutil
import pandas as pd

# ---------------- CONFIG ----------------
CLASSES = ["Garment Lower body", "Socks & Tights", "Underwear"]  # Classes to include
N_PER_CLASS = 10  # Number of images per class
ARTICLES_CSV = "../data/articles.csv"  # Path to your CSV file
ZIP_PATH = "../data/images.zip"  # Path to downloaded zip file
OUTPUT_DIR = "../data/cnn_data/"  # Directory where extracted images will be saved
# ----------------------------------------

# Create output directories for each class
os.makedirs(OUTPUT_DIR, exist_ok=True)  # Create root output folder if it doesn't exist
for c in CLASSES:
    os.makedirs(os.path.join(OUTPUT_DIR, c), exist_ok=True)  # Create subfolder per class

# Load CSV and filter for chosen classes
df = pd.read_csv(ARTICLES_CSV)  # Read articles CSV
df['product_group_name'] = df['product_group_name'].str.strip()  # Remove extra spaces
filtered = df[df['product_group_name'].isin(CLASSES)]  # Keep only rows for selected classes

# Select top N_PER_CLASS articles per class
selected = pd.DataFrame()  # Empty DataFrame to store selected articles
for c in CLASSES:
    subset = filtered[filtered['product_group_name'] == c].head(N_PER_CLASS)  # Take first N_PER_CLASS
    selected = pd.concat([selected, subset])  # Add to selection
selected = selected.reset_index(drop=True)  # Reset row indices

# Zero-pad article_ids to 10 digits to match zip filenames
selected['article_id'] = selected['article_id'].astype(str).str.zfill(10)

# Map article_id to its class
article_class_map = dict(zip(selected['article_id'], selected['product_group_name']))  # Dict: id -> class

# Set of article_ids we need
needed_ids = set(selected['article_id'])  # For quick lookup

# ----------------- Extraction -----------------
with zipfile.ZipFile(ZIP_PATH, 'r') as z:  # Open the zip file
    for member in z.namelist():  # Loop over all files inside the zip
        if not member.lower().endswith('.jpg'):
            continue  # Skip non-image files
        article_id = os.path.splitext(os.path.basename(member))[0]  # Get article ID from filename
        if article_id in needed_ids:  # Only extract if we need it
            cls = article_class_map[article_id]  # Get the class of this article
            dst = os.path.join(OUTPUT_DIR, cls, f"{article_id}.jpg")  # Destination path
            os.makedirs(os.path.dirname(dst), exist_ok=True)  # Ensure folder exists
            with z.open(member) as source, open(dst, "wb") as target:  # Open source & destination
                shutil.copyfileobj(source, target)  # Copy image content

# ----------------- Report -----------------
for c in CLASSES:
    cnt = len([f for f in os.listdir(os.path.join(OUTPUT_DIR, c)) if f.lower().endswith('.jpg')])
    print(f"{c}: {cnt} images copied to {os.path.join(OUTPUT_DIR, c)}")  # Print how many images copied per class


Garment Lower body: 10 images copied to ../data/cnn_data/Garment Lower body
Socks & Tights: 10 images copied to ../data/cnn_data/Socks & Tights
Underwear: 10 images copied to ../data/cnn_data/Underwear


#### Preparing dataset for training,validation and also parameters


In [3]:
import tensorflow as tf  # Import TensorFlow library

# ---------------- Image & dataset parameters ----------------
IMG_HEIGHT = 64  # Height to resize all images to
IMG_WIDTH = 64   # Width to resize all images to
BATCH_SIZE = 32  # Number of images per batch during training
DATA_DIR = '../data/cnn_data/'  # Root folder where your class subfolders are

# ---------------- Create the training dataset ----------------
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,               # Directory containing subfolders per class
    validation_split=0.2,   # Reserve 20% of data for validation
    subset="training",      # Specify this dataset is the training portion
    seed=123,               # Random seed for reproducibility (ensures same split every run)
    image_size=(IMG_HEIGHT, IMG_WIDTH),  # Resize all images to uniform size
    batch_size=BATCH_SIZE   # Number of images per batch
)

# ---------------- Create the validation dataset ----------------
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,               # Same directory as training
    validation_split=0.2,   # Same split percentage
    subset="validation",    # Specify this dataset is the validation portion
    seed=123,               # Same seed to ensure split matches training
    image_size=(IMG_HEIGHT, IMG_WIDTH),  # Resize images to same size as training
    batch_size=BATCH_SIZE   # Same batch size as training
)


Found 30 files belonging to 3 classes.
Using 24 files for training.
Found 30 files belonging to 3 classes.
Using 6 files for validation.


#### Preparation and Training

In [ ]:
model.compile(
    optimizer='adam',  # Specifies the optimization algorithm used to update model weights (Adam is fast and commonly used)
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  
        # Defines the loss function to measure how well the model predicts labels
        # 'SparseCategoricalCrossentropy' is used when labels are integers (not one-hot encoded)
        # 'from_logits=True' means the model's output layer does NOT have a softmax; loss will apply softmax internally
    metrics=['accuracy']  # Metric to track during training; here we monitor accuracy of predictions
)


print("Starting CNN training...")  
# Displays a message indicating that model training is about to begin

history = model.fit(
    train_ds,               # The training dataset containing input images and labels
    validation_data=val_ds, # The validation dataset used to evaluate performance after each epoch
    epochs=5                # Number of times the model will go through the entire training dataset
)
# 'model.fit' trains the CNN on the training data and tracks performance on validation data
# Returns a 'history' object containing training and validation loss and accuracy per epoch
